In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [5]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [6]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [7]:
books["tagged_description"].to_string("tagged_description.txt", 
                                      index=False, 
                                      header=False)

In [8]:
from langchain_community.document_loaders import TextLoader

# Load with UTF-8 encoding
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 50, which is longer than the specified 0
Created a chunk of size 5

In [9]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics ...')

In [15]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [16]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808397 Introduce your baby to birds, cat...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808397 Introduce your baby to birds, cat...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808397 Introduce your baby to birds, cat...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808380 Introduce your babies to birds, c...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808380 Introduce your babies to birds, c...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786808380 Introduce your babies to birds, c...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786851157 Children will delight in hearing ...'),
 Document(metadata={'source': 'tagged_description.txt'}, page_content='9780786851157 Children will delight in hearing ...'),


In [17]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."


In [18]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [19]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
216,9780060760823,0060760826,A Couple of April Fools,Gregory Maguire,Juvenile Fiction,http://books.google.com/books/content?id=qW4cv...,"At a Vermont elementary school, April Fools' D...",2005.0,3.50,240.0,3.0,A Couple of April Fools,"9780060760823 At a Vermont elementary school, ..."
799,9780142003343,0142003344,The Blank Slate,Steven Pinker,Psychology,http://books.google.com/books/content?id=7rJ5g...,In a study of the nature versus nurture debate...,2003.0,4.08,528.0,17851.0,The Blank Slate: The Modern Denial of Human Na...,9780142003343 In a study of the nature versus ...
880,9780152049676,0152049673,Winter is the Warmest Season,Lauren Stringer,Juvenile Fiction,http://books.google.com/books/content?id=S1EOm...,A child describes pleasant ways to stay warm d...,2006.0,3.91,40.0,360.0,Winter is the Warmest Season,9780152049676 A child describes pleasant ways ...
1078,9780241003008,0241003008,The Very Hungry Caterpillar,Eric Carle,Babytime resource,http://books.google.com/books/content?id=DpGEQ...,Eric Carle's children's classic is the story o...,1994.0,4.29,26.0,340101.0,The Very Hungry Caterpillar,9780241003008 Eric Carle's children's classic ...
2387,9780451217769,0451217764,A Certain Justice,John T. Lescroart,Fiction,http://books.google.com/books/content?id=YKLZ6...,A novel that examines the nature of justice--a...,2006.0,4.00,624.0,2921.0,A Certain Justice,9780451217769 A novel that examines the nature...
2827,9780571207565,0571207561,Collected Children's Stories,Sylvia Plath,"Children's poetry, American",http://books.google.com/books/content?id=i6oCG...,"Three classic children's stories, Mrs Cherry's...",2001.0,4.22,54.0,111.0,Collected Children's Stories,9780571207565 Three classic children's stories...
3061,9780679451211,0679451218,The Sibley Field Guide to Birds of Western Nor...,NaN,Nature,http://books.google.com/books/content?id=Fd0PA...,A guide for bird enthusiasts specifically desi...,2003.0,4.70,473.0,709.0,The Sibley Field Guide to Birds of Western Nor...,9780679451211 A guide for bird enthusiasts spe...
3130,9780679847595,0679847596,Ballet Shoes,Noel Streatfeild,Juvenile Fiction,http://books.google.com/books/content?id=0UvIa...,"In a classic children's story, three unsinkabl...",1993.0,4.07,233.0,28222.0,Ballet Shoes,"9780679847595 In a classic children's story, t..."
3296,9780743202411,0743202414,A Primate's Memoir,Robert M. Sapolsky,Nature,http://books.google.com/books/content?id=90Iil...,In the tradition of Jane Goodall and Dian Foss...,2002.0,4.35,304.0,4769.0,A Primate's Memoir: A Neuroscientist's Unconve...,9780743202411 In the tradition of Jane Goodall...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
